In [1]:
!featurize dataset download 910888ea-f616-4cd9-bfd1-4acc754d6fae

Traceback (most recent call last):
  File "/environment/miniconda3/bin/featurize", line 8, in <module>
    sys.exit(cli())
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/environ

In [2]:
!pip install fastai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
!pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
from fastai.tabular.all import *
from fastai.collab import *
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [5]:
# 设置设备，优先使用 MPS，其次 CUDA，最后 CPU
device = torch.device('mps') if torch.backends.mps.is_available() else (
    torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
)
print(f"使用的设备: {device}")

使用的设备: cpu


In [6]:
# Load the data
data_df = pd.read_excel('/home/featurize/[final] April 2015 to Nov 30 2019 - Transformed Jester Data - .xlsx')
  

In [7]:
data_df = pd.DataFrame(data_df)

In [8]:
import pandas as pd  

# 初始化转换后的数据列表  
transformed_data = []  
  
# 遍历每个用户和每个笑话  
for user_id, row in data_df.iterrows():  
    for joke_id, rating in enumerate(row, start=1):  # 使用enumerate来获取索引作为joke_id  
        if rating != 99:  
            transformed_data.append([user_id + 1, joke_id, rating])  # 用户ID和joke_id偏移+1（如果需要）  
  
# 将列表转换为DataFrame  
long_df = pd.DataFrame(transformed_data, columns=['user_id', 'joke_id', 'rating'])  
  
# 显示前几行以验证结果  
print(long_df.head())

   user_id  joke_id  rating
0        1        1    1.00
1        1       73    8.21
2        2        1    4.00
3        2       20   -5.98
4        2       36   -4.47


In [9]:
# Load the jokes
jokes_df = pd.read_excel('/home/featurize/Dataset4JokeSet.xlsx')

In [10]:
# 正文列增加列名joke
jokes_df.columns = ['joke']

In [11]:
#将jokes_df中的索引定义为joke_id
jokes_df = jokes_df.rename_axis('joke_id').reset_index()

In [12]:
#完成实验步骤中对数据集操作后执行
jokes_df.head()

,joke_id,joke
0,0,"This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. ""What could they possibly have said to make you move out?"" ""They told me that you were a pedophile."" He replied, ""That's an awfully big word for a ten year old."""
1,1,Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
2,2,Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.
3,3,"Q.\tWhat's O. J. Simpson's Internet address? A.\tSlash, slash, backslash, slash, slash, escape."
4,4,"Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied, 'No, Bill. If I would have married him, you'd..."


In [13]:
long_df = long_df.merge(jokes_df)

In [14]:
ratings=long_df

In [15]:
# 根据时间戳划分训练集和测试集
split_timestamp = ratings['user_id'].quantile(0.9)  # 按时间戳的90%分位数划分
train_ratings = ratings[ratings['user_id'] <= split_timestamp]
test_ratings = ratings[ratings['user_id'] > split_timestamp]

In [16]:
# 创建数据加载器
dls =CollabDataLoaders.from_df(train_ratings,item_name='joke',bs=64,device=device)

In [17]:
# 训练模型
learn = collab_learner(dls,n_factors=50,y_range=(0,5.5))
learn.model.to(device)  # 将模型移动到指定设备
learn.dls.device = device  # 将数据加载器的设备设为指定设备

In [18]:
# 添加早停和保存最佳模型的回调
callbacks = [
    EarlyStoppingCallback(monitor='valid_loss', patience=3),
    SaveModelCallback(monitor='valid_loss', fname='best_model')
]

In [19]:
learn.fit_one_cycle(100,5e-3,wd=0.1,cbs=callbacks)

epoch,train_loss,valid_loss,time
0,80.397575,72.491890,00:06
1,70.632263,70.534599,00:07
2,62.153793,67.441132,00:12
3,67.255531,65.095367,00:07
4,62.823341,64.165207,00:08
5,44.327236,63.732475,00:08
6,55.336819,63.497631,00:08
7,55.621796,63.342648,00:07
8,58.846794,63.228546,00:08
9,48.166088,63.231628,00:26


Better model found at epoch 0 with valid_loss value: 72.49188995361328.
Better model found at epoch 1 with valid_loss value: 70.53459930419922.
Better model found at epoch 2 with valid_loss value: 67.44113159179688.
Better model found at epoch 3 with valid_loss value: 65.09536743164062.
Better model found at epoch 4 with valid_loss value: 64.16520690917969.
Better model found at epoch 5 with valid_loss value: 63.73247528076172.
Better model found at epoch 6 with valid_loss value: 63.49763107299805.
Better model found at epoch 7 with valid_loss value: 63.342647552490234.
Better model found at epoch 8 with valid_loss value: 63.228546142578125.
No improvement since epoch 8: early stopping


In [20]:
# 在测试集上预测
test_dl = learn.dls.test_dl(test_ratings)
preds, _ = learn.get_preds(dl=test_dl)

In [21]:
# 计算RMSE和MAE
rmse = np.sqrt(mean_squared_error(test_ratings['rating'], preds))
mae = mean_absolute_error(test_ratings['rating'], preds)

In [22]:
print("测试集 RMSE: ", rmse)
print("测试集 MAE: ", mae)

测试集 RMSE:  7.662767328259027
测试集 MAE:  4.63904555709922


In [23]:
# 为新用户选择一个未使用的user_id
new_user_id = ratings['user_id'].max() + 1

In [24]:
# 添加新用户的评分数据
new_ratings_df = pd.DataFrame({
    'user_id': [new_user_id, new_user_id,new_user_id],
        'joke_id': [107,149,34],
        'rating': [10,9,2],
    })
ratings = pd.concat([ratings, new_ratings_df])

In [25]:
# 创建新的数据加载器
dls = CollabDataLoaders.from_df(train_ratings,item_name='joke',bs=64,device=device)

In [26]:
# 重新训练模型
learn = collab_learner(dls,n_factors=50,y_range=(0,5.5))
learn.model.to(device)  # 将模型移动到指定设备
learn.dls.device = device  # 将数据加载器的设备设为指定设备

In [27]:
# 添加早停和保存最佳模型的回调
callbacks = [
    EarlyStoppingCallback(monitor='valid_loss', patience=3),
    SaveModelCallback(monitor='valid_loss', fname='best_model')
]

In [28]:
learn.fit_one_cycle(100,5e-3,wd=0.1,cbs=callbacks)

epoch,train_loss,valid_loss,time
0,71.501656,70.662727,00:09
1,68.919853,68.694626,00:09
2,64.236794,65.416924,00:12
3,57.505791,62.856037,00:06
4,66.014122,61.827850,00:05
5,56.306206,61.399593,00:06
6,53.449211,61.128025,00:07
7,60.769733,60.901428,00:07
8,58.214336,60.846699,00:10
9,46.733139,60.941441,00:07


Better model found at epoch 0 with valid_loss value: 70.66272735595703.
Better model found at epoch 1 with valid_loss value: 68.69462585449219.
Better model found at epoch 2 with valid_loss value: 65.41692352294922.
Better model found at epoch 3 with valid_loss value: 62.85603713989258.
Better model found at epoch 4 with valid_loss value: 61.827850341796875.
Better model found at epoch 5 with valid_loss value: 61.399593353271484.
Better model found at epoch 6 with valid_loss value: 61.12802505493164.
Better model found at epoch 7 with valid_loss value: 60.90142822265625.
Better model found at epoch 8 with valid_loss value: 60.84669876098633.
No improvement since epoch 8: early stopping


In [29]:
# 为新用户生成推荐
joke_ids = ratings['joke_id'].unique()  # 获取所有电影ID列表
joke_ids_new_user = ratings.loc[ratings['user_id'] == new_user_id, 'joke_id']  # 获取新用户评分的电影ID列表
joke_ids_to_pred = np.setdiff1d(joke_ids, joke_ids_new_user)  # 获取新用户未评分的电影ID列表

In [30]:
# 预测新用户未评分电影的评分
testset_new_user = pd.DataFrame({
    'user_id': [new_user_id] * len(joke_ids_to_pred),
    'joke_id': joke_ids_to_pred,
    'joke':jokes_df.loc[jokes_df['joke_id'].isin(joke_ids_to_pred),'joke']
})


In [31]:
test_dl = learn.dls.test_dl(testset_new_user)
preds, _ = learn.get_preds(dl=test_dl)

In [32]:
# 将预测结果添加到testset_new_user DataFrame中
testset_new_user['rating'] = preds.numpy()

In [33]:
# 获取评分最高的前10部电影
top_10_jokes = testset_new_user.nlargest(10, 'rating')

In [34]:
print("新用户的前10部笑话推荐：")
print(top_10_jokes[['joke','rating']])

新用户的前10部笑话推荐：
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        joke  \
1                                                                                                                                                                                                                                                                                                                                                                                         

In [35]:
# 导出训练后的模型
from fastai.collab import *

In [37]:
learn.save('best_model')

Path('models/best_model.pth')

In [38]:
pip install streamlit

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [39]:
import streamlit as st
import random
import pandas as pd

In [40]:
jokes_df = pd.DataFrame({
    "joke_id": [0, 1, 2, 3, 4],  # 笑话的唯一标识符
    "joke": [
        "Why did the chicken cross the road? To get to the other side.",
        "I told my wife she was drawing her eyebrows too high. She looked surprised.",
        "I would tell you a construction joke, but I’m still working on it.",
        "I just Argued with my punctuation teacher. I’m not sure if I made a point.",
        "Did you hear about the guy who invented Lifesavers? They say he made a mint."
    ]
})

In [42]:
# 设置评分的选项
ratings_options = [1, 2, 3, 4, 5]

In [43]:
# 页面标题
st.title("笑话评分应用")
# 随机选择3个笑话显示在界面上
selected_jokes = jokes_df.sample(3)

In [46]:
# 为每个笑话创建一个部分，并添加评分滑块
for i, joke in enumerate(joke):
    st.header(f"笑话 {i+1}")
    st.write(joke)
    # 创建一个评分滑块
    rating = st.slider(f"给笑话 {i+1} 评分", 0, 5, 0)  # 评分范围从0到5
    st.write(f"你的评分: {rating}")
# 提交按钮
submit_button = st.button("提交评分")
if submit_button:
    st.write("感谢您的评分!")

In [47]:
# 定义笑话数据
jokes_db = [
    {
        "joke": "This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. \"What could they possibly have said to make you move out?\" \"They told me that you were a pedophile.\" He replied, \"That's an awfully big word for a ten year old.\"",
        "rating": 0
    },
    {
        "joke": "Just a thought .. Before criticizing someone, walk a mile in their shoes. Then when you do criticize them, you will be a mile away and have their shoes !",
        "rating": 5
    },
    {
        "joke": "An engineer dies and reports to the pearly gates. St. Peter checks his dossier and says, \"Ah, you're an engineer--you're in the wrong place.\" So, the engineer reports to the gates of hell and is let in. Pretty soon, the engineer gets dissatisfied with the level of comfort in hell, and starts designing and building improvements. After awhile, they've got air conditioning, flush toilets and escalators, and the engineer is a pretty popular guy. One day, God calls Satan up on the telephone and says with a sneer, \"So, how's it going down there in hell?\" Satan replies, \"Hey, things are going great. We've got air conditioning, flush toilets and escalators, and there's no telling what this engineer is going to come up with next.\" God replies, \"What?? You've got an engineer? That's a mistake--he should never have gotten down there; send him up here.\" Satan says, \"No way\" I like having an engineer on the staff, and I'm keeping him\" God says, \"Send him back up here or I'll sue\" Satan laughs uproariously and answers, \"Yeah, right. And just where are You going to get a lawyer?\"",
        "rating": 0
    }
]

# 初始用户评分
initial_ratings = {
    "jokes_db[0]['joke']": jokes_db[0]["rating"],
    "jokes_db[1]['joke']": jokes_db[1]["rating"],
    "jokes_db[2]['joke']": jokes_db[2]["rating"]
}


def recommend_jokes(ratings):
    # 假设我们根据笑话的初始评分来推荐
    # 选择评分为5的笑话进行推荐
    recommended_jokes = [joke for joke in jokes_db if joke["rating"] == 5]
    return recommended_jokes

# 页面标题
st.title("笑话评分与推荐")

# 显示并获取初始笑话的评分
for joke in jokes_db:
    st.write(joke["joke"])
    rating = st.slider(f"Rate this joke", 0, 5, initial_ratings.get(joke["joke"], 0))
    st.write(f"Your rating: {rating}")

# 根据评分推荐笑话
recommended_jokes = recommend_jokes(initial_ratings)

# 显示推荐笑话
if recommended_jokes:
    st.header("We recommend the following jokes based on your ratings:")
    for joke in recommended_jokes:
        st.write(joke["joke"])
else:
    st.write("No jokes to recommend based on your ratings.")

# 提供一个提交按钮
submit_button = st.button("Submit Ratings")

if submit_button:
    st.write("Thank you for your ratings!")